# Imports

In [100]:
import pandas as pd
import json
import datetime

#Train/Test section
from sklearn.model_selection import train_test_split

#Scaling the data
from sklearn.preprocessing import StandardScaler

# Data
Import the following data:
- GVB: *Number of travelers per hour at given station coordinates*
- Event Data: *Dates and locations for events in Amsterdam* --> **OUTLIERS**

## GVB

In [18]:
#Read from file
gvb = pd.read_csv("../../../Data_thesis/Sample_data/GVB/ritten.csv")

In [19]:
#Check contents
gvb.head()

,Unnamed: 0,Datum,UurgroepOmschrijving (van vertrek),VertrekHalteCode,VertrekHalteNaam,VertrekXCoord,VertrekYCoord,AantalRitten
0,5,2018-12-24,01:00 - 01:59,5031,Dam,489348393.0,5.237341e+09,58.0
1,8,2018-12-24,01:00 - 01:59,5068,Dam,489124121.0,5.237436e+09,28.0
2,16,2018-12-24,02:00 - 02:59,5031,Dam,489348393.0,5.237341e+09,103.0
3,18,2018-12-24,02:00 - 02:59,5068,Dam,489124121.0,5.237436e+09,25.0
4,26,2018-12-24,03:00 - 03:59,5031,Dam,489348393.0,5.237341e+09,39.0


### Descrive the dataframe
- *Count*: Count number of non-NA/null observations
- *Mean*: Mean of the values.
- *STD*: Standard deviation of the obersvations.
- *Min*: Minimum of the values in the object.
- *25%, 50%, 75%*: Returns the given percentile
- *Max*: Maximum of the values in the object.

In [20]:
gvb.describe()

,Unnamed: 0,VertrekXCoord,VertrekYCoord,AantalRitten
count,972.000000,9.720000e+02,9.720000e+02,972.000000
mean,24975.330247,4.892755e+08,5.237234e+09,203.260288
std,14484.601039,3.871362e+05,1.903516e+05,253.135267
min,5.000000,4.889259e+08,5.236910e+09,10.000000
25%,12567.750000,4.890271e+08,5.237194e+09,40.000000
50%,24419.500000,4.890794e+08,5.237318e+09,94.500000
75%,37513.250000,4.893484e+08,5.237368e+09,271.250000
max,49712.000000,4.901239e+08,5.237493e+09,1371.000000


### Stations in GVB Dataset

In [26]:
stations = gvb.VertrekHalteNaam.unique()

for station in stations:
    print("Station: ", station)

Station:  Dam
Station:  Nieuwmarkt
Station:  Spui


## Events

In [21]:
with open("../Data/Modified/Events.json") as f:
    events = json.load(f)

In [22]:
print(events[0])

{'Event': 'Springsnow Festival', 'Coordinates': {'Latitude': '52,3726380', 'Longtitude': '4,8941060'}, 'Data': {'startdate': '20-04-2018', 'enddate': '20-05-2018'}}


# Model
Below a few models will be instantiated

## GVB Data Only
Below all the models use only the GVB data

### Data Preperation
Make the model with the GVB data from *Dam* station

#### Dataframe formation

In [84]:
#Select all rows with "Dam" as station
dam_df = gvb.loc[gvb["VertrekHalteNaam"] == "Dam"]

#Select usable columns
dam_df = dam_df[["Datum", "UurgroepOmschrijving (van vertrek)", "AantalRitten"]]

#Add columns
#Gives day in numbers --> 0: Monday, 1: Tuesday,..., 6: Sunday
dam_df.insert(2, "weekday", 99)

#Adds whether day is weekend --> Day 5 and 6
dam_df.insert(3, "is_weekend", 0)

#### Transform DF

In [85]:
#Dataframe to dict
dam = dam_df.to_dict("index")

In [86]:
#Loop over dict
for k, v in dam.items():
    
    #Transform the date string to datatime.date object
    date = datetime.date.fromisoformat(v["Datum"])
    
    #Transfrom date to weekday number
    v["weekday"] = date.weekday()
    
    #Check if weekday is in the weekend
    if date.weekday() == 5 or date.weekday() == 6:
        v["is_weekend"] = 1

In [87]:
#Return to dict
dam_df = pd.DataFrame.from_dict(dam, orient="index")

#Remove the "Datum" columns
dam_df = dam_df.drop(columns=["Datum"])

#### Show contents

In [88]:
dam_df.head()

,UurgroepOmschrijving (van vertrek),weekday,is_weekend,AantalRitten
0,01:00 - 01:59,0,0,58.0
1,01:00 - 01:59,0,0,28.0
2,02:00 - 02:59,0,0,103.0
3,02:00 - 02:59,0,0,25.0
4,03:00 - 03:59,0,0,39.0


### Training/Test
Split the dataset into training and test data, using a [function from Sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)

####Train_Test_split Parameters
- *input*: 
    - *x*: data used to predict laballed data
        - Hour slot
        - Weekday (int)
        - Weekend (Boolean)
    - *y*: laballed data
        - Number of rides (int)
- *test_size*: Proportion of total data used as test data, rest is training data
- *random_state*: Number used as seed by the *Random Number Generator* 

In [97]:
#Select only the values from each row
data = dam_df.values

#Split the labels from the rest
x = data[:, :3]
y = data[:, 3]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.1, random_state = 42)

In [99]:
#First three rows train set
x_train[:3]

array([['16:00 - 16:59', 6, 1],
       ['17:00 - 17:59', 0, 0],
       ['23:00 - 23:59', 6, 1]], dtype=object)

### Scale the data
For ML algorithms that use [Euclidean distance between points, large distances between points can pose a problem](https://medium.com/greyatom/why-how-and-when-to-scale-your-features-4b30ab09db5e). 

Scaling is done with the [Standard Scaler from Sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html)

In [103]:
#Call on scaler
scaler = StandardScaler()

#Fit to training data
scaler.fit(x_train[:, 1:])

#Scale the actual data in the training and test set
x_train[:, 1:] = scaler.transform(x_train[:, 1:])
x_test[:, 1:] = scaler.transform(x_test[:, 1:])

D:\Programs\Anaconda\envs\Analytics\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
D:\Programs\Anaconda\envs\Analytics\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
D:\Programs\Anaconda\envs\Analytics\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


### Models
train data on models

#### Random Forrest Regressor
Implement the [Sklearn Version](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html)

Parameters